In [72]:
import sqlite3
import re
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import os
import imaplib
import email
import json
from email.header import decode_header

In [73]:
imap_server = "imap.gmail.com"
IMAP_SERVER = "imap.gmail.com"  # Change for Outlook, Yahoo, etc.
IMAP_PORT = 993

SMTP_SERVER = "smtp.gmail.com"
SMTP_PORT = 587

username= **  # nhi kia env variables set
password = **


# SQLite Database Setup
DB_FILE = "orders.db"

In [74]:

def initialize_db():
    """Create tables if they don't exist."""
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()

    # ✅ Create products table
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS products (
            product_id INTEGER PRIMARY KEY AUTOINCREMENT,
            product_name TEXT NOT NULL,
            category TEXT NOT NULL,
            unit_price REAL NOT NULL
        )
    """)

    # ✅ Create orders table
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS orders (
            order_id INTEGER PRIMARY KEY AUTOINCREMENT,
            customer_name TEXT NOT NULL,
            customer_email TEXT NOT NULL,
            customer_phone TEXT NOT NULL,
            shipping_address TEXT NOT NULL,
            order_date TEXT NOT NULL,
            delivery_date TEXT NOT NULL,
            order_status TEXT NOT NULL
        )
    """)

    # ✅ Create order_items table (To store products in an order)
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS order_items (
            order_item_id INTEGER PRIMARY KEY AUTOINCREMENT,
            order_id INTEGER NOT NULL,
            product_id INTEGER NOT NULL,
            quantity INTEGER NOT NULL,
            FOREIGN KEY (order_id) REFERENCES orders(order_id),
            FOREIGN KEY (product_id) REFERENCES products(product_id)
        )
    """)

    conn.commit()
    conn.close()
    print("✅ Database initialized successfully.")

# Run the function to initialize the DB
initialize_db()


✅ Database initialized successfully.


In [75]:
def check_inbox():
    """Scan inbox for unread order emails and send them to process_order()."""
    try:
        # Connect to Gmail IMAP server
        mail = imaplib.IMAP4_SSL(imap_server)
        mail.login(username, password)

        # Select the inbox
        mail.select("inbox")
        
        # Search for all unread emails
        status, messages = mail.search(None, "UNSEEN")
        if status != "OK":
            print("❌ Error searching emails")
            return
        
        email_ids = messages[0].split()
        print(f"📩 Found {len(email_ids)} unread emails.")
        
        for email_id in email_ids:
            status, msg_data = mail.fetch(email_id, "(RFC822)")
            if status != "OK":
                print("❌ Error fetching email")
                continue

            raw_email = msg_data[0][1]
            msg = email.message_from_bytes(raw_email)

            # Extract subject and sender
            subject, encoding = decode_header(msg["Subject"])[0]
            if isinstance(subject, bytes):
                subject = subject.decode(encoding if encoding else "utf-8")
            
            # Log the subject and sender
            if not subject:
                subject = "[No Subject]"  # Default to "[No Subject]" if the subject is empty
            sender = msg.get("From")
            print(f"Subject: {subject}")
            print(f"From: {sender}")

            # Extract email body (checking for plain text, html, and json)
            email_body = None
            if msg.is_multipart():
                for part in msg.walk():
                    content_type = part.get_content_type()
                    content_disposition = str(part.get("Content-Disposition"))

                    if "attachment" not in content_disposition:
                        if content_type == "text/plain" or content_type == "text/html":
                            email_body = part.get_payload(decode=True).decode()
                            print(f"✅ Found {content_type} content.")
                            break
                        elif content_type == "application/json":
                            email_body = part.get_payload(decode=True).decode()
                            print(f"✅ Found JSON content.")
                            break
            else:
                # If it's not multipart, just get the body
                email_body = msg.get_payload(decode=True).decode()

            # Log the body content (for debugging purposes)
            if email_body:
                print("✅ Email Body Content:\n")
                print(email_body)  # Explicit print for visibility
            else:
                print("❌ No email body found.")

            # Process email body if present
            if email_body:
                try:
                    order_data = json.loads(email_body)  # Parse as JSON
                    process_order(order_data)  # Process the order
                except json.JSONDecodeError:
                    print(f"❌ Invalid JSON format in email from {sender}")
                    send_email(sender, "Order Processing Error", "❌ Your order email format is invalid. Please send JSON.")
            
        # Logout from the server
        mail.logout()

    except Exception as e:
        print(f"❌ Error checking inbox: {e}")

def process_order(order_data):
    """Process the order from email and insert it into the database."""
    required_fields = ['customer_name', 'customer_email', 'customer_phone', 'shipping_address', 'order_date', 'delivery_date', 'products']
    
    # Check if all required fields are present
    if not all(field in order_data for field in required_fields):
        print("❌ Missing required fields in the order data.")
        send_email(order_data['customer_email'], "Order Processing Error", "❌ Your order is missing some required details.")
        return

    # Verify product IDs exist in the database
    valid_product_ids = [p['product_id'] for p in order_data['products']]
    if not validate_products(valid_product_ids):
        print("❌ Invalid product IDs in the order.")
        send_email(order_data['customer_email'], "Order Processing Error", "❌ One or more product IDs are invalid.")
        return

    # Insert order into the database
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()
    
    cursor.execute("""
        INSERT INTO orders (customer_name, customer_email, customer_phone, shipping_address, order_date, delivery_date, product_ids, order_status)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    """, (order_data['customer_name'], order_data['customer_email'], order_data['customer_phone'], order_data['shipping_address'], order_data['order_date'], order_data['delivery_date'], ','.join(map(str, valid_product_ids)), "Pending"))
    
    conn.commit()
    conn.close()
    print("✅ Order successfully added to database.")

def validate_products(product_ids):
    """Validate that all product IDs exist in the products table."""
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()

    cursor.execute("SELECT product_id FROM products WHERE product_id IN ({})".format(','.join(['?']*len(product_ids))), tuple(product_ids))
    existing_product_ids = [row[0] for row in cursor.fetchall()]

    conn.close()

    return set(product_ids).issubset(existing_product_ids)


In [76]:

def send_email(to_email, subject, body):
    """Send email with subject and body."""
    import smtplib
    from email.mime.text import MIMEText
    from email.mime.multipart import MIMEMultipart

    msg = MIMEMultipart()
    msg['From'] = username
    msg['To'] = to_email
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain'))

    try:
        # Connect to the Gmail SMTP server
        with smtplib.SMTP('smtp.gmail.com', 587) as server:
            server.starttls()
            server.login(username, password)
            server.sendmail(username, to_email, msg.as_string())
        print(f"📧 Email sent successfully to {to_email}")
    except Exception as e:
        print(f"❌ Error sending email: {e}")

In [79]:
check_inbox()

📩 Found 1 unread emails.
Subject: god save
From: Atharva Chepe <atharva.chepe16@gmail.com>
✅ Found text/plain content.
✅ Email Body Content:

{
    "customer_name": "John Doe",
    "customer_email": "johndoe@example.com",
    "customer_phone": "+1234567890",
    "shipping_address": "123 Main Street, Springfield, USA",
    "order_date": "2025-02-01",
    "delivery_date": "2025-02-05",
    "order_status": "Pending",
    "products": [
        {
            "product_id": 101,
            "product_name": "Laptop",
            "category": "Electronics",
            "unit_price": 800,
            "quantity": 1
        },
        {
            "product_id": 202,
            "product_name": "Wireless Mouse",
            "category": "Accessories",
            "unit_price": 25,
            "quantity": 2
        }
    ]
}

❌ Invalid product IDs in the order.
📧 Email sent successfully to johndoe@example.com
